In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import pathlib
import os
import numpy as np

In [2]:
pd.set_option('mode.chained_assignment', None)

In [8]:
def plot(lc, a, b, c, new, filename):
    fig, ax = plt.subplots(5, sharex=True, sharey=True)
    fig.set_size_inches(11, 8)
    plt.xlabel('Time')
    plt.ylabel('Magnitude')
    plt.tight_layout()
    
    
    #add columns for removed or not, add change the color for the last plot
    ax[0].errorbar(new.Time, new.Magnitude, new.Error, alpha=0.5, elinewidth=0.1, fmt='.', markersize=5, color='b')
    ax[0].errorbar(c.Time, c.Magnitude, c.Error, alpha=0.5, elinewidth=0.1, fmt='.', markersize=5, color='r')
    ax[1].errorbar(a.Time, a.Magnitude, a.Error, alpha=0.5, elinewidth=0.1, fmt='.', markersize=5)
    ax[2].errorbar(b.Time, b.Magnitude, b.Error, alpha=0.5, elinewidth=0.1, fmt='.', markersize=5)
    ax[3].errorbar(c.Time, c.Magnitude, c.Error, alpha=0.5, elinewidth=0.1, fmt='.', markersize=5)
    ax[4].errorbar(new.Time, new.Magnitude, new.Error, alpha=0.5, elinewidth=0.1, fmt='.', markersize=5)
    
    ax[0].set_title("Original LC")
    ax[1].set_title("Anomalies by RF")
    ax[2].set_title("Anomalies by moving frame")
    ax[3].set_title("Anomalies in both")
    ax[4].set_title("New LC")
    
    
    for ax in fig.get_axes():
        ax.label_outer()
        ax.invert_yaxis()
    
    path = "D:\\Astro\\plots\\" 
    path = path + filename + '.pdf'
    fig.savefig(path)
    plt.close('all')

In [14]:
def modelStuff(lc, filename):
    #add the random forest to not go over the parts with more outliers and then lower contamination
    model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.25, max_features=1.0)
    model.fit(lc)
    lc['anomaly_rf'] = pd.Series(model.predict(lc))
    
    lc['step'] = abs(lc['Magnitude'].diff(periods = 1))
    lc['next_step'] = abs(lc['Magnitude'].diff(periods = 2))
    lc['next_step'] = lc['next_step'].shift(-1)
    
    lc['step'] = lc['step'].fillna(1)
    lc['next_step'] = lc['next_step'].fillna(1)    
    
    lc['anomaly_step'] = ((lc['step'] > (lc['next_step'])) & (lc['step'] > (lc['Error']*0.075)))
    
    a = lc.loc[(lc['anomaly_rf'] == -1), ['Time', 'Magnitude', 'Error']]
    b = lc.loc[(lc['anomaly_step'] == True), ['Time', 'Magnitude', 'Error']]
    
    c= lc.loc[(lc['anomaly_rf'] == -1) & (lc['anomaly_step'] == True), ['Time', 'Magnitude', 'Error']]
    new = lc[~lc.index.isin(c.index)]
    
    plot(lc, a, b, c, new, filename)
    
    newPath = "D:\\Astro\\new_LCs\\" + "new_" + filename + ".lc"
    lc.to_csv((newPath), index=False)      

In [5]:
init_outliers = pd.read_csv("C:\\Users\\grant\\OneDrive\\Desktop\\School\\Astro\\reaction_wheel_desaturation_events.txt")
init_outliers = init_outliers.drop(['image_filename', 'DQUALITY'], axis=1)
print(init_outliers.dtypes)
print(init_outliers.head())

camera         int64
ccd            int64
timestart    float64
timeend      float64
dtype: object
   camera  ccd    timestart      timeend
0       1    1  1327.839240  1327.860073
1       1    1  1330.339247  1330.360081
2       1    1  1332.839250  1332.860083
3       1    1  1335.339237  1335.360071
4       1    1  1337.839221  1337.860054


In [6]:
def initial_drops(init_outliers, lc, filename):
    ccd_val = int(filename[-1])
    cam_val = int(filename[-3])
    init = len(lc.index)
    a = init_outliers.loc[(init_outliers['ccd']==ccd_val) & (init_outliers['camera']==cam_val), ['timestart', 'timeend']]
    for index, row in a.iterrows():
        lc = lc.loc[(lc['Time'] > getattr(row, 'timeend')) | (lc['Time'] < getattr(row, 'timestart'))]
    final = len(lc.index)
    print("# of removed outliers: " + str((init-final)))
    return lc

In [ ]:
cols=['Time', 'Magnitude', 'Error']
# path = pathlib.Path("C:\\Users\\grant\\OneDrive\\Desktop\\School\\Astro\\389475380_sector01_1_1.lc")
# lc = pd.read_csv(r"C:\\Users\\grant\\OneDrive\\Desktop\\School\\Astro\\29753242_sector01_1_4.lc", header=None, sep=" ")
# lc.columns=cols
# filename = "8195216_sector01_1_1"
# lc = initial_drops(init_outliers, lc, filename)
# modelStuff(lc, filename)

for filename in os.listdir("D:\\Astro\\_tess_sector01_lc\\"):
      if filename.endswith(".lc"):
          path = pathlib.Path("D:\\Astro\\_tess_sector01_lc\\" + filename)
          lc = pd.read_csv(path, header=None, sep=" ")
          lc.columns=cols 
          filename = filename[:-3]
          lc = initial_drops(init_outliers, lc, filename)
          modelStuff(lc, filename)
      else:
          continue

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

...

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed

# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed outliers: 15
# of removed